In [ ]:
!pip install pinecone-client
from pinecone import Pinecone, ServerlessSpec

Setup access to notebook with api key

In [ ]:
pckey = userdata.get('pinecone')
pc = Pinecone(api_key=pckey)

Create an index

In [ ]:
index_name = "rag"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    while not pc.describe_index(index_name).status["ready"]:
      time.sleep(1)

index = pc.Index(index_name)

**Chromadb to pandas df**

In [ ]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
### NOTE dim size is 1024 NOT 768 ########
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
persist_directory = 'fulldb_cliuserguide'
!unzip "/content/drive/MyDrive/Colab Notebooks/My NN stuff/alteon/chroma_db_cliuserguide.zip"
vectordb_cliguide = Chroma(persist_directory=persist_directory, embedding_function=hf)
print(vectordb_cliguide._collection.count())


top = vectordb_cliguide.get(include=['uris','documents','metadatas', 'embeddings'])
newdict = {}
newdict['ids'] = top['ids']
newdict['metadata'] = top['metadatas']
newdict['docs'] = top['documents']
newdict['values']= top['embeddings']

df = pd.DataFrame(newdict)
df = df.rename(columns={"docs":"name"}, errors="raise")
df['id'] = ["cliguides#chunk"+str(i) for i in range(1, len(df) + 1)]
print (df.columns.to_list()) # ['id', 'metadatas', 'docs', 'embedding']
texts = df['name'].to_dict()
metadata = df['metadata'].to_dict()
for i in metadata:
  metadata[i]['text']=texts[i]
df['metadata']=[metadata[i] for i in metadata]

df= df[['id','values','metadata']]

**Push dataframe to Pinecone**

In [ ]:
index_name = "rag"
index = pc.Index(index_name)
index.upsert_from_dataframe(df, namespace = 'ALTEON_CLI_GUIDE')

**query index stats**

In [ ]:
index.describe_index_stats()

**Query and semantic search**

In [ ]:
question = "How to disable management access to data ports?"

query_results = index.query(
    namespace='ALTEON_CLI_GUIDE',
    vector=hf.embed_query(question) ,
    top_k=5,
    include_values=False,
    include_metadata = True
)
print(query_results)

{'matches': [{'id': 'cliguides#chunk194',
              'metadata': {'source': 'revised alteon cli guide.pdf',
                           'text': 'Document ID: RDWR-ALOS-V3358_CLIAG2403\n'
                                   '\n'
                                   '57\n'
                                   '\n'
                                   'Alteon Command Line Interface Application '
                                   'Guide Accessing Alteon\n'
                                   '\n'
                                   '>> Management Port # apply\n'
                                   '\n'
                                   '(Make your changes active)\n'
                                   '\n'
                                   '>> Management Port # cur\n'
                                   '\n'
                                   '(View current settings)\n'
                                   '\n'
                                   '>> Management Port # save\n'
                       